# Лабораторная работа № 2. Предсказание открытия срочного депозита

Выполнил Голдобин Илья N34523

Цель работы: Основываясь на данных кампаний прямого маркетинга португальского банковского учреждения научиться предсказывать откроет ли клиент депозит

Задачи работы: 
1. Построить алгоритм предсказания открытия клиентом срочного депозита.
2. Оценить точность прогноза.

## Обработка данных

Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np

Загружаем данные из файла

In [2]:
df_bank = pd.read_csv('bank.csv', sep=';')
df_bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


Рассмотрим уникальные значения в столбце "education"

In [3]:
df_bank['education'].unique()

array(['primary', 'secondary', 'tertiary', 'unknown'], dtype=object)

Рассмотрим сколько раз встречаются уникальные значения в столбце "education"

In [4]:
df_bank['education'].value_counts()

secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64

Количество значений "unknown" назначительно относительно общего числа наблюдений, поэтому можем удалить их

In [5]:
df_bank = df_bank.drop(df_bank[df_bank['education'] == 'unknown'].index)
df_bank['education'].value_counts()

secondary    2306
tertiary     1350
primary       678
Name: education, dtype: int64

Перекодируем текстовые значения столбца "education" в числовые категории

In [6]:
df_bank['education'] = df_bank['education'].map({'primary': 1, 'secondary': 2, 'tertiary': 3})
df_bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,1,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,2,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,3,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,3,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,2,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,3,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,2,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,2,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


Рассмотрим уникальные значения в столбце "default"

In [7]:
df_bank['default'].unique()

array(['no', 'yes'], dtype=object)

Перекодируем их

In [8]:
df_bank['default'] = df_bank['default'].map({'no': 0, 'yes': 1})
df_bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,1,0,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,2,0,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,3,0,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,3,0,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,2,0,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,3,1,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,2,0,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,2,0,1137,no,no,cellular,6,feb,129,4,211,3,other,no


Рассмотрим уникальные значения в столбце "housing"

In [9]:
df_bank['housing'].unique()

array(['no', 'yes'], dtype=object)

Перекодируем их

In [10]:
df_bank['housing'] = df_bank['housing'].map({'no': 0, 'yes': 1})
df_bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,1,0,1787,0,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,2,0,4789,1,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,3,0,1350,1,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,3,0,1476,1,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,2,0,0,1,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,1,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,3,1,-3313,1,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,2,0,295,0,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,2,0,1137,0,no,cellular,6,feb,129,4,211,3,other,no


Рассмотрим уникальные значения в столбце "loan"

In [11]:
df_bank['loan'].unique()

array(['no', 'yes'], dtype=object)

Перекодируем их

In [12]:
df_bank['loan'] = df_bank['loan'].map({'no': 0, 'yes': 1})
df_bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,1,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,2,0,4789,1,1,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,3,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,3,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,2,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,1,0,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,3,1,-3313,1,1,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,2,0,295,0,0,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,2,0,1137,0,0,cellular,6,feb,129,4,211,3,other,no


Рассмотрим уникальные значения в столбце "contact"

In [13]:
df_bank['contact'].unique()

array(['cellular', 'unknown', 'telephone'], dtype=object)

Рассмотрим сколько раз встречаются уникальные значения в столбце "contact"

In [14]:
df_bank['contact'].value_counts()

cellular     2801
unknown      1251
telephone     282
Name: contact, dtype: int64

Слишком много неизвестных значений в столбце, не ясно как заменить их, и это может негативно сказаться на точности модели, поэтому стоит удалить столбец

In [15]:
df_bank = df_bank.drop('contact', axis=1)
df_bank

,age,job,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,1,0,1787,0,0,19,oct,79,1,-1,0,unknown,no
1,33,services,married,2,0,4789,1,1,11,may,220,1,339,4,failure,no
2,35,management,single,3,0,1350,1,0,16,apr,185,1,330,1,failure,no
3,30,management,married,3,0,1476,1,1,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,2,0,0,1,0,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,1,0,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,3,1,-3313,1,1,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,2,0,295,0,0,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,2,0,1137,0,0,6,feb,129,4,211,3,other,no


Перекодируем столбец day. Разделим значения на 2 категории: первая половина месяца и вторая половина

In [16]:
# df_bank.loc[df_bank['day'] <= 7, 'day'] = 1
# df_bank.loc[((7 < df_bank['day']) & (df_bank['day'] <= 14)), 'day'] = 2
# df_bank.loc[((14 < df_bank['day']) & (df_bank['day'] <= 21)), 'day'] = 3
# df_bank.loc[((21 < df_bank['day']) & (df_bank['day'] <= 31)), 'day'] = 4

# logreg 0.2 dectree 0.18

df_bank.loc[df_bank['day'] <= 15, 'day'] = 1
df_bank.loc[((15 < df_bank['day']) & (df_bank['day'] <= 31)), 'day'] = 2

# logreg 0.19 dectree 0.2

Проверим

In [17]:
df_bank['day'].value_counts()

2    2251
1    2083
Name: day, dtype: int64

Рассмотрим квартили по столбцу balance 

In [18]:
df_bank.describe()

,age,education,default,balance,housing,loan,day,duration,campaign,pdays,previous
count,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000
mean,40.991924,2.155053,0.016844,1410.637517,0.571297,0.157822,1.519382,264.544301,2.806876,39.670974,0.544070
std,10.505378,0.666325,0.128700,3010.612091,0.494948,0.364616,0.499682,260.642141,3.129682,99.934062,1.702219
min,19.000000,1.000000,0.000000,-3313.000000,0.000000,0.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,2.000000,0.000000,67.000000,0.000000,0.000000,1.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,2.000000,0.000000,440.000000,1.000000,0.000000,2.000000,186.000000,2.000000,-1.000000,0.000000
75%,48.000000,3.000000,0.000000,1464.000000,1.000000,0.000000,2.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,3.000000,1.000000,71188.000000,1.000000,1.000000,2.000000,3025.000000,50.000000,871.000000,25.000000


Перекодируем столбце balance в соответствии с значениями квартилей

In [19]:
# df_bank.loc[df_bank['balance'] <= 66.5, 'balance'] = 1
# df_bank.loc[((66.5 < df_bank['balance']) & (df_bank['balance'] <= 440)), 'balance'] = 2
# df_bank.loc[((440 < df_bank['balance']) & (df_bank['balance'] <= 1456)), 'balance'] = 3
# df_bank.loc[((1456 < df_bank['balance']) & (df_bank['balance'] <= 71188)), 'balance'] = 4
# 0.19 0.19

Столбец campaign содержит количество контактов с клиентом в течение данной кампании, столбец previous содержит количество контактов за прошлые рекламные кампании. Можно совместить столбцы

In [20]:
df_bank['previous'] += df_bank['campaign']
df_bank = df_bank.drop('campaign', axis=1)
df_bank
# если закомментировать, то logreg хуже на 0.04, dectree лучше на 0.01

,age,job,marital,education,default,balance,housing,loan,day,month,duration,pdays,previous,poutcome,y
0,30,unemployed,married,1,0,1787,0,0,2,oct,79,-1,1,unknown,no
1,33,services,married,2,0,4789,1,1,1,may,220,339,5,failure,no
2,35,management,single,3,0,1350,1,0,2,apr,185,330,2,failure,no
3,30,management,married,3,0,1476,1,1,1,jun,199,-1,4,unknown,no
4,59,blue-collar,married,2,0,0,1,0,1,may,226,-1,1,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,1,0,2,jul,329,-1,5,unknown,no
4517,57,self-employed,married,3,1,-3313,1,1,1,may,153,-1,1,unknown,no
4518,57,technician,married,2,0,295,0,0,2,aug,151,-1,11,unknown,no
4519,28,blue-collar,married,2,0,1137,0,0,1,feb,129,211,7,other,no


Столбец pdays содержит количество дней, прошедших с предыдушего контакта с клиентом. В случае отсутствия контактов, значение -1. Более подходящим для создания модели будет замена значения -1 на либо очень большое значение, либо на максимальное значение столбца.

In [21]:
df_bank.loc[df_bank['pdays'] == -1, 'pdays'] = df_bank['pdays'].unique().max()
# df_bank.loc[df_bank['pdays'] == -1, 'pdays'] = =999 =9999 -99999 -999999

Проверим резлуьтаты замены

In [22]:
df_bank['pdays'].value_counts()

871    3556
182      23
183      19
363      11
91       11
       ... 
404       1
118       1
386       1
81        1
234       1
Name: pdays, Length: 284, dtype: int64

Воспользуемся функцией get_dummies, чтобы разложить столбец "month" на 12 столбцов, каждый из которых соответствует одному месяцу

In [23]:
df_bank = pd.get_dummies(df_bank, columns=['month'])
# df_bank = df_bank.drop('month_dec', axis=1)
df_bank

,age,job,marital,education,default,balance,housing,loan,day,duration,...,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,30,unemployed,married,1,0,1787,0,0,2,79,...,0,0,0,0,0,0,0,0,1,0
1,33,services,married,2,0,4789,1,1,1,220,...,0,0,0,0,0,0,1,0,0,0
2,35,management,single,3,0,1350,1,0,2,185,...,0,0,0,0,0,0,0,0,0,0
3,30,management,married,3,0,1476,1,1,1,199,...,0,0,0,0,1,0,0,0,0,0
4,59,blue-collar,married,2,0,0,1,0,1,226,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,1,0,2,329,...,0,0,0,1,0,0,0,0,0,0
4517,57,self-employed,married,3,1,-3313,1,1,1,153,...,0,0,0,0,0,0,1,0,0,0
4518,57,technician,married,2,0,295,0,0,2,151,...,0,0,0,0,0,0,0,0,0,0
4519,28,blue-collar,married,2,0,1137,0,0,1,129,...,0,1,0,0,0,0,0,0,0,0


Столбец содержащий длительность звонка слишком сильно влияет на результирующую переменную. Кроме того, длительность звонка становится известной только после принятия решения о том, будет ли продукт подписан. Поэтому для построения более реалистичной модели рекомендуется удалить данный столбец

In [24]:
df_bank = df_bank.drop('duration', axis=1)
df_bank

,age,job,marital,education,default,balance,housing,loan,day,pdays,...,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,30,unemployed,married,1,0,1787,0,0,2,871,...,0,0,0,0,0,0,0,0,1,0
1,33,services,married,2,0,4789,1,1,1,339,...,0,0,0,0,0,0,1,0,0,0
2,35,management,single,3,0,1350,1,0,2,330,...,0,0,0,0,0,0,0,0,0,0
3,30,management,married,3,0,1476,1,1,1,871,...,0,0,0,0,1,0,0,0,0,0
4,59,blue-collar,married,2,0,0,1,0,1,871,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,1,0,2,871,...,0,0,0,1,0,0,0,0,0,0
4517,57,self-employed,married,3,1,-3313,1,1,1,871,...,0,0,0,0,0,0,1,0,0,0
4518,57,technician,married,2,0,295,0,0,2,871,...,0,0,0,0,0,0,0,0,0,0
4519,28,blue-collar,married,2,0,1137,0,0,1,211,...,0,1,0,0,0,0,0,0,0,0


Столбец "poutcome" содержит данные о результатах прошлого звонка. Рассмотрим сколько раз встречаются уникальные значения в этом столбце

In [25]:
df_bank['poutcome'].value_counts()

unknown    3555
failure     471
other       189
success     119
Name: poutcome, dtype: int64

В описании данных указано, что для столбца "poutcome" предусмотрены значения failure, nonexistent, success. Однако в нашем случае есть еще значение "other".
В столбце много значений "unknown", поэтому удалить все записи с этим значением нельзя. Можно приравнять значения "unknown" и "other" к "failure", т. к. по сути это будет означать, что ранее клиент не брал услугу у банка. Исходя из этого перекодируем столбец таким образом, что все, что "success" = 1, а все остальное - 0

In [26]:
df_bank['poutcome'] = df_bank['poutcome'].map({'unknown': 0, 'failure': 0, 'other': 0, 'success': 1}) # gbm лучше
# df_bank = pd.get_dummies(df_bank, columns=['poutcome'])
df_bank

,age,job,marital,education,default,balance,housing,loan,day,pdays,...,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,30,unemployed,married,1,0,1787,0,0,2,871,...,0,0,0,0,0,0,0,0,1,0
1,33,services,married,2,0,4789,1,1,1,339,...,0,0,0,0,0,0,1,0,0,0
2,35,management,single,3,0,1350,1,0,2,330,...,0,0,0,0,0,0,0,0,0,0
3,30,management,married,3,0,1476,1,1,1,871,...,0,0,0,0,1,0,0,0,0,0
4,59,blue-collar,married,2,0,0,1,0,1,871,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,2,0,-333,1,0,2,871,...,0,0,0,1,0,0,0,0,0,0
4517,57,self-employed,married,3,1,-3313,1,1,1,871,...,0,0,0,0,0,0,1,0,0,0
4518,57,technician,married,2,0,295,0,0,2,871,...,0,0,0,0,0,0,0,0,0,0
4519,28,blue-collar,married,2,0,1137,0,0,1,211,...,0,1,0,0,0,0,0,0,0,0


Рассмотрим значения в столбце job

In [27]:
df_bank['job'].value_counts()

management       942
blue-collar      905
technician       746
admin.           461
services         404
retired          216
self-employed    179
entrepreneur     157
unemployed       126
housemaid        107
student           68
unknown           23
Name: job, dtype: int64

Количество значений unknown мало, поэтому их можно удалить

In [28]:
df_bank = df_bank.drop(df_bank[df_bank['job'] == 'unknown'].index)

Проверим удаление записей с job == unknown

In [29]:
df_bank['job'].unique()

array(['unemployed', 'services', 'management', 'blue-collar',
       'self-employed', 'technician', 'entrepreneur', 'admin.', 'student',
       'housemaid', 'retired'], dtype=object)

Записи удалены

Столбец martial содержит семейное положение, столбец job работу клиента. Трудно перекодировать эти значения, поэтому можно воспользоваться функцией get_dummies, чтобы разложить эти столбцы

In [30]:

df_bank = pd.get_dummies(df_bank, columns=['job', 'marital'])
# df_bank = pd.get_dummies(df_bank, columns=['marital'])
# df_bank['job'] = df_bank['job'].map({'unemployed': 0, 'services': 1, 'managment': 2.5, 'blue-collar': 1, 'self-employed': 1.5})
# df_bank['job'] = df_bank['job'].map({'techinican': 2, 'entrepreneur': 2.5, 'admin.': 2.7, 'student': 0, 'housemaid': 1, 'retired': 1})
# df_bank = df_bank.drop('job', axis=1)
# logreg хуже на 0.3
df_bank

,age,education,default,balance,housing,loan,day,pdays,previous,poutcome,...,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single
0,30,1,0,1787,0,0,2,871,1,0,...,0,0,0,0,0,0,1,0,1,0
1,33,2,0,4789,1,1,1,339,5,0,...,0,0,0,1,0,0,0,0,1,0
2,35,3,0,1350,1,0,2,330,2,0,...,1,0,0,0,0,0,0,0,0,1
3,30,3,0,1476,1,1,1,871,4,0,...,1,0,0,0,0,0,0,0,1,0
4,59,2,0,0,1,0,1,871,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,2,0,-333,1,0,2,871,5,0,...,0,0,0,1,0,0,0,0,1,0
4517,57,3,1,-3313,1,1,1,871,1,0,...,0,0,1,0,0,0,0,0,1,0
4518,57,2,0,295,0,0,2,871,11,0,...,0,0,0,0,0,1,0,0,1,0
4519,28,2,0,1137,0,0,1,211,7,0,...,0,0,0,0,0,0,0,0,1,0


Перекодируем результирующий столбец

In [31]:
df_bank['y'] = df_bank['y'].map({'no': 0, 'yes': 1})

In [32]:
# from sklearn import preprocessing

# norm = preprocessing.MinMaxScaler()
# norm.fit(df_bank)
# df_bank_norm = norm.transform(df_bank)
# df_bank_norm = pd.DataFrame(df_bank_norm, index = df_bank.index, columns = df_bank.columns)

Разделим данные на признаки и результат

In [33]:
y = df_bank['y']
X = df_bank.drop('y', axis=1)

Проверим полученные в результате разделения данные

In [34]:
X.describe()

,age,education,default,balance,housing,loan,day,pdays,previous,poutcome,...,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single
count,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.00000,...,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000,4311.000000
mean,40.970309,2.155648,0.016933,1410.662723,0.574113,0.158432,1.519137,754.979819,3.350267,0.02714,...,0.218511,0.050104,0.041522,0.093714,0.015774,0.173046,0.029228,0.116678,0.617954,0.265368
std,10.500185,0.665448,0.129037,3015.650889,0.494534,0.365188,0.499692,252.713903,3.467486,0.16251,...,0.413284,0.218186,0.199517,0.291464,0.124613,0.378331,0.168463,0.321074,0.485944,0.441580
min,19.000000,1.000000,0.000000,-3313.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,2.000000,0.000000,66.500000,0.000000,0.000000,1.000000,871.000000,1.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,39.000000,2.000000,0.000000,440.000000,1.000000,0.000000,2.000000,871.000000,2.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,48.000000,3.000000,0.000000,1456.000000,1.000000,0.000000,2.000000,871.000000,4.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,87.000000,3.000000,1.000000,71188.000000,1.000000,1.000000,2.000000,871.000000,50.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
y.describe()

count    4311.000000
mean        0.115518
std         0.319684
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: y, dtype: float64

С данными все в порядке, записи не потеряны

In [36]:
y.value_counts()

0    3813
1     498
Name: y, dtype: int64

В реузльтирующем столбце неравное отношение между двумя классами. Нам важно, чтобы при разделении выборки на тренировочную и тестирующую оказалось достаточное количество записей с разным результатом

Импортируем функцию для разделения выборки на тестовую и тренировочную. Возьмем 1/5 часть выборки как тестовую, остальное пойдет в тренировочную выборку. Используем параметр stratify=y, чтобы в обеих выборках оказалась примерно равная доля положительных и отрицательных значений y

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify=y)
# random_state = 42 - для воспроизводимости результатов на разных компьютерах (деление происходит случайным образом)
# test_size = 0.2 - объем тестовой выборки

Рассмотрим как разделились значения y по выборкам

In [38]:
y_test.value_counts()

0    763
1    100
Name: y, dtype: int64

In [39]:
y_train.value_counts()

0    3050
1     398
Name: y, dtype: int64

Значения разделилсь равномерно

## Логистическая регрессия


Импортируем функцию для создания модели логистической регрессии и выбираем параметры

In [40]:
from sklearn.linear_model import LogisticRegression
model_lg = LogisticRegression(
    # метод для поиска решения. Для небольших датасетов лучше подходит liblinear, sag и saga - для больших.
    # Варианты: newton-cg, lbfgs, liblinear, sag, saga
    solver = 'newton-cg',
    # норма для регуляризации. Варианты: 'l1', 'l2', 'elasticnet', 'none'
    penalty = 'none',
    # параметр регуляризации. Чем меньше, тем сильнее регуляризация. Положительный.
    C = 1,
    # параметр для остановки поиска решения.
    tol = 1e-4,
    # Опция для задачи мультиклассификации, когда есть несколько классов.
    multi_class = 'ovr'
    )

С помощью функции fit рассчитаем параметры модели на тренировочных данных

In [41]:
model_lg.fit(X_train, y_train)

LogisticRegression(C=1, multi_class='ovr', penalty='none', solver='newton-cg')

На основе полученной модели и тестовой выборки совершим расчет результата

In [42]:
y_pred_lg = model_lg.predict(X_test)

Рассмотрим коэффициенты модели

In [43]:
model_lg.coef_

array([[-4.55566838e-03,  1.33999265e-01,  4.86523122e-01,
        -1.80260463e-05, -2.14705592e-01, -4.93231137e-01,
         2.98337158e-02, -9.02695871e-04, -3.43884208e-02,
         2.22379699e+00,  1.20874104e-01, -2.77157862e-01,
         8.64647970e-01, -3.17952403e-01, -6.18432286e-01,
        -4.15724034e-01, -2.56033766e-01,  1.05403169e+00,
        -8.33667341e-01, -7.85121130e-01,  9.63257753e-01,
        -4.63953152e-02, -1.25006007e-01, -3.47244964e-01,
         5.97007770e-02, -2.96879575e-02, -7.13103437e-02,
         5.64902583e-01,  6.14381218e-02, -2.89657350e-01,
         5.18050113e-01, -2.85953452e-01, -6.02904137e-01,
         4.60720024e-02, -4.46335102e-01, -1.47409517e-01]])

In [44]:
X.columns

Index(['age', 'education', 'default', 'balance', 'housing', 'loan', 'day',
       'pdays', 'previous', 'poutcome', 'month_apr', 'month_aug', 'month_dec',
       'month_feb', 'month_jan', 'month_jul', 'month_jun', 'month_mar',
       'month_may', 'month_nov', 'month_oct', 'month_sep', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single'],
      dtype='object')

Используем функцию оценки качества для оценки точности работы модели. Наиболее вероятной целью банка будет нахождение клиентов, готовых взять услугу у банка, поэтому будем оценивать модель по recall для "1".

In [45]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred_lg))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       763
           1       0.61      0.19      0.29       100

    accuracy                           0.89       863
   macro avg       0.76      0.59      0.62       863
weighted avg       0.87      0.89      0.87       863



Вероятность, что клиент, который по мнению модели должен взять услугу, действительно ее возьмет составляет 19%

## Дерево решений

Импортируем функцию для создания модели дерева решений, выбираем параметры дерева, обучаем ее на тренировной выборке

In [46]:
from sklearn.tree import DecisionTreeClassifier # Импортируем метод

# Инициализируем модель
model_dt = DecisionTreeClassifier(random_state = 42, criterion = 'gini', max_depth = 5, min_samples_leaf = 10)

# random_state = 42, для воспроизводимости результатов 
# criterion = 'gini', критерий разбиения ('gini' или 'entropy')
# max_depth = 3, максимальная глубина дерева
# min_samples_leaf = 5, минимальное число элементов в листе (может быть долей)

# Обучаем модель
model_dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=10, random_state=42)

Рассмотрим какие признаки модель считает важными 

In [47]:
pd.DataFrame({'feature': X.columns,
              'importance': model_dt.feature_importances_}).sort_values('importance', ascending = False)

,feature,importance
9,poutcome,0.480574
0,age,0.113018
7,pdays,0.093417
20,month_oct,0.083071
3,balance,0.071996
6,day,0.063737
17,month_mar,0.048263
4,housing,0.044531
1,education,0.001394
27,job_retired,0.000000


Наиболее важным явялется факт того, что клиент брал ранее эту услугу в ходе прошлых маркетинговых кампаний. Интересно, что 4 по важности признаком является то, был ли последний контакт с клиентом в рамках текущей маркетинговой кампании совершен в октябре

Применим модель на тестовых данных

In [48]:
y_pred_dt = model_dt.predict(X_test)

Рассмотрим точночность работы обученной модели. 

In [49]:
print(metrics.classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94       763
           1       0.69      0.20      0.31       100

    accuracy                           0.90       863
   macro avg       0.80      0.59      0.63       863
weighted avg       0.88      0.90      0.87       863



Рассмотрим показатель recall. Обученная модель предсказала 20 из 100 клиентов в тестовой выборке

Перейдем к ансамблям моделей

## Ансамбли моделей 

### Случайный лес

Импортируем функцию создания моедли случайного леса, вызываем ее и настраиваем параметры. Обучаем модель на тренировочных данных и оцениваем ее точность на тестовых данных

In [50]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state = 42, 
                                  n_estimators = 100, 
                                  criterion ='gini', 
                                  max_depth = 20
                                 ) # max features

model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
print(metrics.classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94       763
           1       0.61      0.17      0.27       100

    accuracy                           0.89       863
   macro avg       0.75      0.58      0.60       863
weighted avg       0.87      0.89      0.86       863



Рассмотрим важность признаков

In [51]:
pd.DataFrame({'feature': X.columns,
              'importance': model_rf.feature_importances_}).sort_values('importance', ascending = False)

,feature,importance
3,balance,0.187775
0,age,0.160292
8,previous,0.086257
7,pdays,0.086195
9,poutcome,0.062152
1,education,0.034506
6,day,0.032633
4,housing,0.028128
34,marital_married,0.017899
31,job_technician,0.017264


В отличие от дерева решений, наиболее важным признаком является то, сколько денег на счете у клиента. Также, хоть и уже не таким важным, является то, связывались ли с клиентом в последний раз в октябре.

Случайный лес показал себя хуже на тестовой выборке, угадав 17 из 100 потенциальных клиентов, важность признаков у него более равномерно распределена, чем у модели дерева решений. Возможно случайный лес покажет себя лучше при других тестовых данных, чем дерево решений

### GBM

Импортируем функцию для создания модели градиентного бустинга.
Вызываем ее и поределяем параметры модели. 
Обучаем модель на тренировочных данных.
Тестируем модель на тестовых данных.
Оцениваем точность модели.

In [52]:
from sklearn.ensemble import GradientBoostingClassifier


model = GradientBoostingClassifier(random_state = 42,
                                   # Число деревьев, участвующих в приближении решения
                                   n_estimators = 500,
                                   # Максимальная глубина каждого дерева
                                   max_depth = 2,
                                   # Параметр, уменьшающий переобучение, являющемся весом отдельного дерева.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate = 0.3,
                                   # Есть и другие параметры, уменьшающие размер дерева,
                                   # такие же как у DecisionTreeClassifier
                                   )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.94       763
           1       0.63      0.31      0.42       100

    accuracy                           0.90       863
   macro avg       0.77      0.64      0.68       863
weighted avg       0.88      0.90      0.88       863



Точнсть модели значительно возросла по сравнению с случайным лесом, деревом решений и линейной регрессией.

### XGBoost

Импортируем функцию для создания модели XGBoost, зададим параметры модели, обучим ее на тренировочной выборке и рассмотрим точность работы обученной модели на тестовой выборке

In [53]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state = 42,
                      n_estimators = 1200,
                      max_depth = 3,
                      learning_rate = 0.3,
                      eval_metric = 'merror',
                      tree_method = 'approx',
                      scale_pos_weight = 8
                     )
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print (metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       763
           1       0.44      0.37      0.40       100

    accuracy                           0.87       863
   macro avg       0.68      0.65      0.66       863
weighted avg       0.86      0.87      0.87       863



In [54]:
pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending = False)

,feature,importance
9,poutcome,0.100664
19,month_nov,0.052801
17,month_mar,0.041620
20,month_oct,0.040938
10,month_apr,0.037716
18,month_may,0.036086
27,job_retired,0.034175
5,loan,0.034087
32,job_unemployed,0.031834
7,pdays,0.030356


Модель XGBoost смогла опознать 37 потенциальных клиентов из 100, но в то же время точность модели стала меньше по сравнению с ранее рассмотренными. В рамках задачи это не так важно, так как в приоритете стоит поиск клиентов

### CatBoost

Импортируем функцию для создания CatBoost модели

In [55]:
from catboost import CatBoostClassifier


Вызываем функцию для создания модели и выбираем ее параметры. Обучаем модель на тренировочной выборке и применяемм ее на тестовой выборке, далее оцениваем метрики качества сделанных предсказаний для тестовых данных

In [56]:
# cat_features = ['balance', 'poutcome', 'previous', 'pdays']
# cat_features = [i for i, x in enumerate(X.columns) if x in cat_features]
model = CatBoostClassifier(random_seed = 42,
                           iterations = 1000,
                           depth = 5,
                           learning_rate = 0.25,
                           verbose = False,
                          )
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       763
           1       0.56      0.30      0.39       100

    accuracy                           0.89       863
   macro avg       0.73      0.63      0.66       863
weighted avg       0.87      0.89      0.88       863



Модели удалось угадать 30 клиентов из 100

Рассмотрим важность признаков клиентов по мнению модели

In [57]:
pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending = False)

,feature,importance
3,balance,23.937967
0,age,16.759729
8,previous,9.555180
7,pdays,4.818818
1,education,4.734971
6,day,3.011048
4,housing,2.855718
23,job_blue-collar,2.368458
11,month_aug,2.343014
18,month_may,2.223878


В отличии от дерева решений и случайного леса, наиболее важным фактором модель посчитала баланс и возраст. Действительно, баланс кажется более важным фактором в случае принятия решения об открытии срочного депозита.

### Вывод

В ходе проделанной работы была решена задача классификации на основе реальных данных о клиентах португальского банка.
Задача решалась такими методами, как: линейная регрессия, деревья решений, случайны лес, GBM, XGBoost, CatBoost.
Основной метрикой качества при оценивании модели была выбрана полнота (recall) выбора клиентов, которые возьмут услугу у банка.
Было обнаружено, что использование бустинга дает значительный прирост в эффективности для нашей задачи по сравнению с простыми моделями и даже ансамблями (случайный лес). Тем не менее даже различные техники значительно различаются между собой. Наиболее эффективной оказался метод XGBoost, благодаря которому удалось обучить модель, которая предсказала 37% потенциальных клиентов.
Невысокие результаты можно объяснить тем, что анализируемый датасет явялется лишь частью оригинального, в нем отсутствуют некоторые признаки и часть записей о клиентах. Используя полный набор данных удалось бы увеличить точность работы моделей.